In [13]:
#%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


__Data Saved:__

- __{PATH_FOREYE}/correTS_{uid}.csv__ --> corrected ET + HT coordinates + calculated velocities
- __{PATH_FOREYE}/interval_mad_{uid}.csv__ --> periods in which saccade threshold is calculated + ET, HT thresholds

# Preprocessing

In [3]:
import copy  # copy big/deep objects by value
import csv
import datetime  # datetime operations
import itertools  # operate with iterators
import json  # read/write from/into json format
import math
import os  # OS operations (read/write files/folders)
import sys
import time
import warnings  # hide warnings
from collections import Counter
from itertools import groupby
import matplotlib

# process parallelization
from multiprocessing import Manager, Pool, RawArray, cpu_count
from os.path import exists

import matplotlib.pyplot as plt  # mother of plots focr Python

# import mlxtend
import numpy as np  # array/matrix operations (e.g. linear algebra)
import pandas as pd  # operate with dataframes
import pyxdf  # read XDF files (LSL streams recordings)

import scipy.stats
import seaborn as sns  # matplotlib plotting nice with shortcuts
from IPython.display import Markdown, display  # print nicely
from ipywidgets import IntProgress
#from matplotlib.pyplot import cm
from scipy.signal import savgol_coeffs
from tqdm.notebook import tqdm, trange  # mother of progressbars
from scipy.stats import ks_2samp

In [4]:
# warnings.simplefilter(action="ignore", category=FutureWarning)

# raw and processed data paths
PATH_RAW = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/27.07.23/data"
PATH_PROC = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/Events"
PATH_FOREYE = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/ET_Output_MAD-sacc"
PATH_TRG = "C:/Users/schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/EEG_Data_Skripte_Debbie/TriggerFiles_fEEG"


# specify decimals format on pandas tables
pd.options.display.float_format = "{:.3f}".format

# inline static plotting (default)
%matplotlib inline
# interactive plotting
# %matplotlib widget

# progress bar customized format
B_FORMAT = """📄 {n_fmt} of {total_fmt} {desc} processed: {bar} 
            {percentage:3.0f}% ⏱️{elapsed} ⏳{remaining} ⚙️{rate_fmt}{postfix}"""


CORES = cpu_count()  # number of cpu threads for multiprocessing
print(f"Total CPU threads: {CORES}")


def pbar_fork_hack():
    """
    Hack to enforce progress bars to be displayed by fork processes on
    IPython Apps like Jupyter Notebooks.

    Avoids [IPKernelApp] WARNING | WARNING: attempted to send message from fork

    Important: pass this function as argument for the initializer parameter
    while initializing a multiprocessing pool to make it work. E.g.:

    pool = Pool(processes=N_CORES, initializer=pbar_fork_hack)

    Source:
     - https://github.com/ipython/ipython/issues/11049#issue-306086846
     - https://github.com/tqdm/tqdm/issues/485#issuecomment-473338308
    """
    print(" ", end="", flush=True)

Total CPU threads: 32


In [5]:
recordings = pd.read_csv("C:/Users/Schmi/Documents/PhD_Osnabruck_University/SpaRe-VR/Spare-VR-EEG/RecordingInfoPilot1_expl.csv", index_col=0)
recordings


,file,created,length,start,old_id
new_id,,,,,
18d7a183-6a4a-4778-bd24-e74a388ffaaa,sub-P1_ses-S1_task-Default_run-001_eeg.xdf,2023-07-27 12:56:00,"34' 5""",1192539.221,18d7a183-6a4a-4778-bd24-e74a388ffaaa


In [6]:
# This will be used to select which version to use
# 1 == MAD_woBig # this is the version we should use (data-driven)
# 2 == 10_sec
define_intervals = 1

# Function at_mad

In [7]:
# Function: calculate MAD saccade
def at_mad(angular_vel, th_0=200):
    # defines the saccade threshold (code from Ashima)
    threshs = []
    while True:
        # take th_0
        threshs.append(th_0)
        # get all angles smaller than this
        angular_vel = angular_vel[angular_vel < th_0]

        # MAD:
        # take the median of all angles smaller than th_0
        median = np.median(angular_vel)
        # substract the median value
        diff = np.sqrt((angular_vel - median) ** 2)
        # get the median of these values
        med_abs_deviation = np.median(diff)

        # calcualte the next threshold with the median
        # 1.486 used when assuming a normal distribution
        th_1 = median + 3 * 1.486 * med_abs_deviation
        # if the thresholds are too different, redo the while loop
        if abs(th_0 - th_1) > 1:
            th_0 = th_1
        # else, set the final threshold to the current one, break the while loop and return values
        else:
            saccade_thresh = th_1
            threshs.append(saccade_thresh)
            break
    return saccade_thresh, threshs

# Done with new folder! Preprocess Data, Interpolate Blinks

In [6]:
### Final Adjustment Blinks
def generate_for_eye(uid):
    """
    Correct is an event is too small.

    Parameters:
        uid (str): Participant identifier.
    """

    # replacement for nans
    nanrep = np.nan

    # to determin the area around the blinks
    min_blink_duration = 0.02
    dilate_nan = 0.023  # add it as two samples before and after blink onset # In paper: 0.01 (<1 sample), therefore we use 0.023 (2 samples)

    # get the cooridnates
    hit_sort = pd.read_csv(f"{PATH_PROC}/Behavior_new_{uid}.csv", index_col=0)
    hit_pos = pd.read_csv(f"{PATH_PROC}/HitsSorted_new_{uid}.csv", index_col=0)

    # delete identical rows:
    hit_sort = hit_sort[~hit_sort.index.duplicated(keep="first")]
    hit_pos = hit_pos[~hit_pos.index.duplicated(keep="first")]

    # ETW_direction:
    x_coord = hit_sort["eyeDirectionCombinedWorld.x"].tolist()
    y_coord = hit_sort["eyeDirectionCombinedWorld.y"].tolist()
    z_coord = hit_sort["eyeDirectionCombinedWorld.z"].tolist()
    # ETW_origin:
    x_coord_orig = hit_sort["eyePositionCombinedWorld.x"].tolist()
    y_coord_orig = hit_sort["eyePositionCombinedWorld.y"].tolist()
    z_coord_orig = hit_sort["eyePositionCombinedWorld.z"].tolist()
    # ETL_direction:
    x_local_dir = hit_sort["eyeDirectionCombinedLocal.x"].tolist()
    y_local_dir = hit_sort["eyeDirectionCombinedLocal.y"].tolist()
    z_local_dir = hit_sort["eyeDirectionCombinedLocal.z"].tolist()
    # HT_direction:
    x_head = hit_sort["hmdDirectionForward.x"].tolist()
    y_head = hit_sort["hmdDirectionForward.y"].tolist()
    z_head = hit_sort["hmdDirectionForward.z"].tolist()
    # HT_origin:
    x_head_orig = hit_sort["hmdPosition.x"].tolist()
    y_head_orig = hit_sort["hmdPosition.y"].tolist()
    z_head_orig = hit_sort["hmdPosition.z"].tolist()
    # HitPos
    hpoo_x = hit_pos["HitPointOnObject.x"].tolist()
    hpoo_y = hit_pos["HitPointOnObject.y"].tolist()
    hpoo_z = hit_pos["HitPointOnObject.z"].tolist()
    # hon
    hon = hit_pos["hitObjectColliderName"].tolist()
    # HitObjectPosition
    hop_x = hit_pos["ObjectColliderBoundsCenter.x"].tolist()
    hop_y = hit_pos["ObjectColliderBoundsCenter.y"].tolist()
    hop_z = hit_pos["ObjectColliderBoundsCenter.z"].tolist()
    
    # and get the timestamps
    times = hit_sort.index.tolist()
    times_hpoo = hit_pos.index.tolist()
    
    # get a list with True == valid (so eyes open) and False == invalid (so eyes closed/not detected)
    filt = ((hit_sort["gazeValidityCLR.Combined"] != 8) & 
            ((hit_sort["eyeOpennessLR.LeftEye"] >= 0.05) | (hit_sort["eyeOpennessLR.RightEye"] >= 0.05)) & 
            ((hit_sort["LeftEye_mm"] != -1) | (hit_sort["RightEye_mm"] != -1)))

    #hit_sort['valid'] = filt
    valid = filt.tolist()

    # to check how many close samples there were
    cnt = False
    cnt_item = 0.0

    # to get the blink onset
    blinking = False  # will be true during the blinking
    blink_time = 0.0  # will be updated to blink onset
    blinks = [0.0] * len(times)
   

    # times progress bar
    times_pbar = tqdm(
        iterable=times,
        desc=f"⌚ timestamps from participant {uid}",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )
    
    # go through the entire list of timestamps
    for t, item in enumerate(times_pbar):
        # if this is not part of the hpoo:
        if not item in times_hpoo:
            hpoo_x.insert(t, nanrep)
            hpoo_y.insert(t, nanrep)
            hpoo_z.insert(t, nanrep)
            hon.insert(t, nanrep)
            hop_x.insert(t, nanrep)
            hop_y.insert(t, nanrep)
            hop_z.insert(t, nanrep)
            times_hpoo.insert(t, item)

        # to check for blinks: get blink onset
        if (valid[t] == False) and not blinking:
            blinking = True
            # get the time of blinking onset
            blink_time = item
        # if they are over, do:
        elif valid[t] == True and blinking:
            # reset the blinking parameter
            blinking = False
            # get the end of the blink (so the previous time stamp)
            it = times[t - 1]
            # if the blink duration is bigger than the min blink duration:
            # adapt the blink duration plus the dialate_nan
            if it - blink_time >= min_blink_duration:
                # we want to exchange all the items with nan, so get the time interval we need to change it in:
                ts = times[
                    times.index(
                        list(
                            filter(
                                lambda i: i >= (blink_time - dilate_nan), times
                            )
                        )[0]
                    ) : times.index(
                        list(filter(lambda i: i <= (it + dilate_nan), times))[
                            -1
                        ]
                    )
                    + 1
                ]  # get all timestamps in the important time window
                for t_s in ts:
                    # if this is not part of the hpoo (as we add two elements after the current one):
                    if not t_s in times_hpoo:
                        hpoo_x.insert(times.index(t_s), nanrep)
                        hpoo_y.insert(times.index(t_s), nanrep)
                        hpoo_z.insert(times.index(t_s), nanrep)
                        hon.insert(times.index(t_s), nanrep)
                        hop_x.insert(times.index(t_s), nanrep)
                        hop_y.insert(times.index(t_s), nanrep)
                        hop_z.insert(times.index(t_s), nanrep)
                        times_hpoo.insert(times.index(t_s), item)

                    # ETW
                    x_coord[times.index(t_s)] = nanrep
                    y_coord[times.index(t_s)] = nanrep
                    z_coord[times.index(t_s)] = nanrep
                    # ETW_orig
                    x_coord_orig[times.index(t_s)] = nanrep
                    y_coord_orig[times.index(t_s)] = nanrep
                    z_coord_orig[times.index(t_s)] = nanrep
                    # ETL
                    x_local_dir[times.index(t_s)] = nanrep
                    x_local_dir[times.index(t_s)] = nanrep
                    x_local_dir[times.index(t_s)] = nanrep
                    # HT_dir
                    x_head[times.index(t_s)] = nanrep
                    y_head[times.index(t_s)] = nanrep
                    z_head[times.index(t_s)] = nanrep
                    # HT_orig
                    x_head_orig[times.index(t_s)] = nanrep
                    y_head_orig[times.index(t_s)] = nanrep
                    z_head_orig[times.index(t_s)] = nanrep
                    # hpoo
                    hpoo_x[times.index(t_s)] = nanrep
                    hpoo_y[times.index(t_s)] = nanrep
                    hpoo_z[times.index(t_s)] = nanrep
                    hon[times.index(t_s)] = nanrep
                    # hop
                    hop_x[times.index(t_s)] = nanrep
                    hop_y[times.index(t_s)] = nanrep
                    hop_z[times.index(t_s)] = nanrep
                    # add the number to blinks
                    blinks[times.index(t_s)] = nanrep
            # if the blink duration is too small, we do not add an additional window around it
            else:
                ts = times[
                    times.index(
                        list(filter(lambda i: i >= (blink_time), times))[0]
                    ) : times.index(
                        list(filter(lambda i: i <= (it), times))[-1]
                    )
                    + 1
                ]  # get all timestamps in the important time window
                for t_s in ts:
                    # ETW
                    x_coord[times.index(t_s)] = nanrep
                    y_coord[times.index(t_s)] = nanrep
                    z_coord[times.index(t_s)] = nanrep
                    # ETW_orig
                    x_coord_orig[times.index(t_s)] = nanrep
                    y_coord_orig[times.index(t_s)] = nanrep
                    z_coord_orig[times.index(t_s)] = nanrep
                    # ETL
                    x_local_dir[times.index(t_s)] = nanrep
                    x_local_dir[times.index(t_s)] = nanrep
                    x_local_dir[times.index(t_s)] = nanrep
                    # HT_dir
                    x_head[times.index(t_s)] = nanrep
                    y_head[times.index(t_s)] = nanrep
                    z_head[times.index(t_s)] = nanrep
                    # HT_orig
                    x_head_orig[times.index(t_s)] = nanrep
                    y_head_orig[times.index(t_s)] = nanrep
                    z_head_orig[times.index(t_s)] = nanrep
                    # hpoo
                    hpoo_x[times.index(t_s)] = nanrep
                    hpoo_y[times.index(t_s)] = nanrep
                    hpoo_z[times.index(t_s)] = nanrep
                    hon[times.index(t_s)] = nanrep
                    # hop
                    hop_x[times.index(t_s)] = nanrep
                    hop_y[times.index(t_s)] = nanrep
                    hop_z[times.index(t_s)] = nanrep
    

    
    # go through the other list and delete all elmenets that are not in time
    # so all the elements that we cannot add the the .csv like this
    to_del = list(set(times_hpoo) - set(times))
    for i in to_del:
        hpoo_x.pop(times_hpoo.index(i))
        hpoo_y.pop(times_hpoo.index(i))
        hpoo_z.pop(times_hpoo.index(i))
        times_hpoo.pop(times_hpoo.index(i))
        hop_x.pop(times_hpoo.index(i))
        hop_y.pop(times_hpoo.index(i))
        hop_z.pop(times_hpoo.index(i))

    # hon: interpolate if there is only one nan between the same colliders
    for h in range(len(hon) - 2):
        if hon[h + 2] == hon[h] and pd.isnull(hon[h + 1]):
            hon[h + 1] = hon[h]
    # hon: now get the first only collider
    new_col = [
        hon[n] if hon[n] != hon[n - 1] and not pd.isnull(hon[n]) else np.nan
        for n in range(len(hon))
    ]


    # create a df out of all the important lists:
    for_eye = list(
        zip(
            times,
            valid,
            x_coord,
            y_coord,
            z_coord,
            x_coord_orig,
            y_coord_orig,
            z_coord_orig,
            x_local_dir,
            y_local_dir,
            z_local_dir,
            x_head,
            y_head,
            z_head,
            x_head_orig,
            y_head_orig,
            z_head_orig,
            hpoo_x,
            hpoo_y,
            hpoo_z,
            new_col,
            hon,
            hop_x,
            hop_y,
            hop_z,
            blinks,
        )
    )
    for_eye = pd.DataFrame(
        for_eye,
        columns=[
            "time",
            "valid",
            "eyeDirectionCombinedWorld.x",
            "eyeDirectionCombinedWorld.y",
            "eyeDirectionCombinedWorld.z",
            "eyePositionCombinedWorld.x",
            "eyePositionCombinedWorld.y",
            "eyePositionCombinedWorld.z",
            "eyeDirectionCombinedLocal.x",
            "eyeDirectionCombinedLocal.y",
            "eyeDirectionCombinedLocal.z",
            "hmdDirectionForward.x",
            "hmdDirectionForward.y",
            "hmdDirectionForward.z",
            "hmdPosition.x",
            "hmdPosition.y",
            "hmdPosition.z",
            "HitPointOnObject.x",
            "HitPointOnObject.y",
            "HitPointOnObject.z",
            # VS: This is like HitObjectColliderName_all but subsequent hits on the same collider are nan
            "hitObjectColliderName",
            "hitObjectColliderName_all", 
            "ObjectColliderBoundsCenter.x",
            "ObjectColliderBoundsCenter.y",
            "ObjectColliderBoundsCenter.z",
            "blinks",
        ],
    )
    for_eye.set_index("time")
    
    # interpolate
    for column_name in for_eye:
        # do not interpolate these columns
        if column_name not in [
            "time",
            "valid",
            # "blinks",
            "hon",
            "hon_all",
        ]:
            b = for_eye[column_name].values.tolist()
            # get number of nan
            v = [
                len(list(group))
                for key, group in groupby(b, key=pd.isnull)
                if key
            ]

            # get corresponding time for each group in v
            idx = [
                idx + 1
                for idx in range(len(b) - 1)
                if not pd.isnull(b[idx]) and pd.isnull(b[idx + 1])
            ]
            if pd.isnull(b[0]):
                idx.insert(
                    0, 0
                )  # if the first element is nan, it will be added here

            # interpolate data
            for_eye[column_name] = for_eye[column_name].interpolate(
                method="linear", limit_direction="both"
            )
            # go through v: if the beginning and end difference is bigger than allowed, replace interpolated data with nan
            b = for_eye[column_name].values.tolist()
            b = np.array(
                b
            )  # for the filling in an array is needed instead of a list
            for t, item in enumerate(idx):
                # finish for the last timestamp
                if item + v[t] == len(times):
                    break
                # if the distance is bigger then 250ms we do not want to interpolate --> replace values with nan
                if times[item + v[t]] - times[item] > 0.25:
                    b[item : item + v[t]] = np.nan * len(b[item : item + v[t]])

            # replace the column with interpolated one
            for_eye[column_name] = b.tolist()

    
    # save df:
    for_eye.to_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index=True)
    
    
# participants ids
ids = recordings.index.tolist()
idd = ids[:]

# participants progress bar
parts_pbar = tqdm(
    iterable=idd,
    total=len(idd),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

# loop necessary for displaying properly the progressbar with multiprocessing
# source: https://stackoverflow.com/a/40133278
for i in parts_pbar:
    generate_for_eye(i)

📄 0 of 1 📂 participants processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

📄 0 of 159252 ⌚ timestamps from participant 18d7a183-6a4a-4778-bd24-e74a388ffaaa processed:            
      …

# Done with new folder! Smoothing the Data: 5-point median Filter

In [7]:
# To use it for later!!!
# based on remodnav --> has almost the same length as our filter
#ids = recordings.index.tolist()
idd = ids[:]

for i, uid in enumerate(idd):
    display(uid)
    for_eye = pd.read_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index_col=0)

    # the coordinates to be smoothed
    Xcorr_position_old = for_eye["eyePositionCombinedWorld.x"].tolist()
    Ycorr_position_old = for_eye["eyePositionCombinedWorld.y"].tolist()
    Zcorr_position_old = for_eye["eyePositionCombinedWorld.z"].tolist()
    hpooX_old = for_eye["HitPointOnObject.x"].tolist()
    hpooY_old = for_eye["HitPointOnObject.y"].tolist()
    hpooZ_old = for_eye["HitPointOnObject.z"].tolist()

    Xcorr_position = []
    Ycorr_position = []
    Zcorr_position = []
    hpooX = []
    hpooY = []
    hpooZ = []
    # smooth it:
    for s in range(len(Xcorr_position_old)):
        if s - 2 > 0 and s + 2 <= len(Xcorr_position_old):
            Xcorr_position.append(
                np.nanmedian([Xcorr_position_old[s - 2 : s + 3]])
            )
            Ycorr_position.append(
                np.nanmedian([Ycorr_position_old[s - 2 : s + 3]])
            )
            Zcorr_position.append(
                np.nanmedian([Zcorr_position_old[s - 2 : s + 3]])
            )

            hpooX.append(np.nanmedian([hpooX_old[s - 2 : s + 3]]))
            hpooY.append(np.nanmedian([hpooY_old[s - 2 : s + 3]]))
            hpooZ.append(np.nanmedian([hpooZ_old[s - 2 : s + 3]]))

        elif s - 2 < 0:
            Xcorr_position.append(np.nanmedian([Xcorr_position_old[: s + 3]]))
            Ycorr_position.append(np.nanmedian([Ycorr_position_old[: s + 3]]))
            Zcorr_position.append(np.nanmedian([Zcorr_position_old[: s + 3]]))

            hpooX.append(np.nanmedian([hpooX_old[: s + 3]]))
            hpooY.append(np.nanmedian([hpooY_old[: s + 3]]))
            hpooZ.append(np.nanmedian([hpooZ_old[: s + 3]]))
        else:
            Xcorr_position.append(np.nanmedian([Xcorr_position_old[s - 2 :]]))
            Ycorr_position.append(np.nanmedian([Ycorr_position_old[s - 2 :]]))
            Zcorr_position.append(np.nanmedian([Zcorr_position_old[s - 2 :]]))

            hpooX.append(np.nanmedian([hpooX_old[s - 2 :]]))
            hpooY.append(np.nanmedian([hpooY_old[s - 2 :]]))
            hpooZ.append(np.nanmedian([hpooZ_old[s - 2 :]]))

    for_eye_n = pd.read_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index_col=0)
    for_eye["eyePositionCombinedWorld.x"] = Xcorr_position
    for_eye["eyePositionCombinedWorld.y"] = Ycorr_position
    for_eye["eyePositionCombinedWorld.z"] = Zcorr_position
    for_eye["HitPointOnObject.x"] = hpooX
    for_eye["HitPointOnObject.y"] = hpooY
    for_eye["HitPointOnObject.z"] = hpooZ
    for_eye.to_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:28: RuntimeWarning: All-NaN slice encountered
  np.nanmedian([Xcorr_position_old[s - 2 : s + 3]])
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:31: RuntimeWarning: All-NaN slice encountered
  np.nanmedian([Ycorr_position_old[s - 2 : s + 3]])
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:34: RuntimeWarning: All-NaN slice encountered
  np.nanmedian([Zcorr_position_old[s - 2 : s + 3]])
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:37: RuntimeWarning: All-NaN slice encountered
  hpooX.append(np.nanmedian([hpooX_old[s - 2 : s + 3]]))
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:38: RuntimeWarning: All-NaN slice encountered
  hpooY.append(np.nanmedian([hpooY_old[s - 2 : s + 3]]))
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1211250631.py:39: RuntimeWarning: All-NaN slice encountered
  hpooZ.append(np.nanmedian([hpooZ_old[s - 2 : s + 3]]))


# Caluclate angles & velocities

## Calculate the eye-tracking veloctiy

In [8]:
# p.s.
#
# v_gaze_vec(t) = [Xhpoo(t) - Xhpoo(t-1), Yhpoo(t) - Yhpoo(t-1), Zhpoo(t) - Zhpoo(t-1)]
#
# Now we project this velocity onto the plane orthogonal to the viewing
# direction at time t and determine the length. For simplicity we assume
# that all vertical distances are small and keep the plane vertically
# oriented.
#
# v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||
#
# where <> indicates the scalar product
# gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in
# world coordinates.
# v_gaze_inplane is a scalar indicating the velocity in world coordinates
# at the location that is gazed at orthogonal to the gaze axis.
# || ... || indicates the euclidian norm
#
# w_gaze(t) = arctan2(distance(subject(i), hpoo(t)), v_gaze_inplane)
#
# distance(subject(i), hpoo(t)) could be equivalently written as ||
# subject_vec(t) - hpoo_vec(t)||
#
# w_gaze is the instantaneous angular velocity of the eye movement and
# what we want to know.

In [9]:
columns = [
    "time",
    "valid",
    "eyeDirectionCombinedWorld.x",
    "eyeDirectionCombinedWorld.y",
    "eyeDirectionCombinedWorld.z",
    "eyePositionCombinedWorld.x",
    "eyePositionCombinedWorld.y",
    "eyePositionCombinedWorld.z",
    "eyeDirectionCombinedLocal.x",
    "eyeDirectionCombinedLocal.y",
    "eyeDirectionCombinedLocal.z",
    "hmdDirectionForward.x",
    "hmdDirectionForward.y",
    "hmdDirectionForward.z",
    "hmdPosition.x",
    "hmdPosition.y",
    "hmdPosition.z",
    "HitPointOnObject.x",
    "HitPointOnObject.y",
    "HitPointOnObject.z",
    # VS: This is like HitObjectColliderName_all but subsequent hits on the same collider are nan
    "hitObjectColliderName",
    "hitObjectColliderName_all", 
    "ObjectColliderBoundsCenter.x",
    "ObjectColliderBoundsCenter.y",
    "ObjectColliderBoundsCenter.z",
    "blinks",
]

#ids = recordings.index.tolist()
idd = ids[:]


for i, uid in enumerate(idd):
    display(uid)
    # load data:
    for_eye = pd.read_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index_col=0)
    for_eye = for_eye[columns]

    time = for_eye["time"].tolist()
    ts = time

    # get individual coordinates
    # position
    Xcorr_position = for_eye["eyePositionCombinedWorld.x"].tolist()
    Ycorr_position = for_eye["eyePositionCombinedWorld.y"].tolist()
    Zcorr_position = for_eye["eyePositionCombinedWorld.z"].tolist()
    subj = list(zip(Xcorr_position, Ycorr_position, Zcorr_position))

    # hpoo
    hpooX = for_eye["HitPointOnObject.x"].tolist()
    hpooY = for_eye["HitPointOnObject.y"].tolist()
    hpooZ = for_eye["HitPointOnObject.z"].tolist()
    hpoo = list(zip(hpooX, hpooY, hpooZ))

    # v_gaze_vec: get difference in hpoo
    v_vX = pd.DataFrame(hpooX).apply(lambda x: x.diff())[0].tolist()
    v_vY = pd.DataFrame(hpooY).apply(lambda x: x.diff())[0].tolist()
    v_vZ = pd.DataFrame(hpooZ).apply(lambda x: x.diff())[0].tolist()

    # get difference in time:
    ts = pd.DataFrame(time).apply(lambda x: x.diff())[0].tolist()

    v_gaze_vec = list(zip(v_vX, v_vY, v_vZ))

    # gaze_vec(t) is a unit vector in the direction of the gaze (eye+head) in world coordinates
    g_vec = [np.array(hpoo[v] - np.array(subj[v])) for v in range(len(subj))]
    gaze_vec = [np.array(v) / np.linalg.norm(np.array(v)) for v in g_vec]
    # display(np.linalg.norm(gaze_vec[1]))

    # v_gaze_inplane: is a scalar indicating the velocity in world coordinates at the location that is gazed at orthogonal to the gaze axis.
    # v_gaze_inplane(t) = ||v_gaze_vec(t) - (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))||:
    # z1 = (<v_gaze_vec(t), gaze_vec(t)>)
    z1 = [
        np.array(v_gaze_vec[t]).dot(np.array(gaze_vec[t]))
        for t in range(len(v_vX))
    ]

    # z = (<v_gaze_vec(t), gaze_vec(t)> * gaze_vec(t))
    z = [z1[t] * np.array(gaze_vec[t]) for t in range(len(v_vX))]

    # ||v_gaze_vec(t) - z||
    v_gaze_inplane = [
        np.linalg.norm((np.array(v_gaze_vec[t]) - z[t]).tolist())
        for t in range(len(v_gaze_vec))
    ]
    # w_gaze(t) = arctan2(||subject_vec(t) - hpoo_vec(t)||, v_gaze_inplane)

    # sub_hpoo = ||subject_vec(t) - hpoo_vec(t)||
    sub_hpoo = [
        np.linalg.norm(np.array(subj[t]) - np.array(hpoo[t]))
        for t in range(len(hpoo))
    ]

    # arctan2(v_gaze_inplane, sub_hpoo)
    w_gaze = np.arctan2(v_gaze_inplane, sub_hpoo).tolist()

    # turn angle of radians into degrees
    w_gaze = [(w / ts[idx] * 180 / math.pi) for idx, w in enumerate(w_gaze)]
    # w_gaze = [(w*180/math.pi) for idx,w in enumerate(w_gaze)]

    # save df
    for_eye["combined_vel"] = w_gaze
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True
        )

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

In [10]:
### MAXIMUM VELOCITY THRESHOLD (1000deg/sec)
#ids = recordings.index.tolist()
idd = ids[:]

max_vel = 1000.0  # max_vel threshold

for i, uid in enumerate(idd):
    display(uid)
    # load data:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    combined_vel = for_eye["combined_vel"].tolist()
    c_v = [max_vel if cv > max_vel else cv for cv in combined_vel]
    for_eye["combined_vel"] = c_v

    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

# Smooth the Data (Savitzky–Golay)

In [11]:
import scipy.signal

# Taken from NYSTRÖM AND HOLMQVIST, 2010
# Filter: Savitzky–Golay (sgolay in MATLAB)
# Filter order: 2
# Filter length: 2* min saccade duration Peak

ids = recordings.index.tolist()
idd = ids[:]

# define parameters
savgol_length = 3  # 0.01 * 90 #window size = int(savgol_length * sr); sr = 90; remodnav = 0.019 -->  int(savgol_length * sr) not possible with our SR
savgol_polyord = 2  # polynomial order


for i, uid in enumerate(idd):
    display(uid)
    # load data: common timestream for x-ticks
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    combined_vel = for_eye["combined_vel"].tolist()

    # filter
    combined_vel = scipy.signal.savgol_filter(
        combined_vel, savgol_length, savgol_polyord
    )

    # save data
    for_eye["combined_vel"] = combined_vel
    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)


'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

# Calculate the data segmenation intervals

## Exclude big saccs

In [12]:
# Calcualte the threshold across the entire dataset

# determining a critical velocity on a median-filtered (median filter length) time series comprising
# the full duration of a recording. All such periods of consecutive above-threshold velocities are
# weighted by the sum of these velocities. Boundaries of time series chunks are determined by selecting
# such events sequentially (starting with the largest sums) until a maximum average frequency across
# the whole time series is reached resulting chunks represent data intervals between saccades of maximum
# magnitude in the respective data.

ids = recordings.index.tolist()
idd = ids[:]

for i, uid in enumerate(idd):
    display(uid)
    for_eye = pd.read_csv(
        f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
    )
    # get time
    time = for_eye["time"].tolist()

    # run MAD saccade on the entire data
    saccade_th, thres = at_mad(for_eye["combined_vel"])
    # detect all periods above the calculated threshold
    fast_sacc = [
        ti if ti >= saccade_th else np.nan
        for ti in for_eye["combined_vel"].tolist()
    ]

    interv = [np.nan] * len(time)  # rem
    interv[0] = 600  # rem
    total_v = [np.nan] * len(time)  # rem
    start = [time[0]]  # onset time
    start_idx = [0]  # onset idx
    end = []  # offset time
    end_idx = []  # offset index
    total_vels = []  # save the velocities to sort the fast_sacc

    # to add and update for each interval
    starts = []  # save the indicies in
    vels = []  # save the velocities in
    cnt = True  # every time it will be set to true, add the velocity

    for v in range(1, (len(fast_sacc) - 1)):
        # if you start an event higher than the threshold:
        if pd.isnull(fast_sacc[v]) and not pd.isnull(fast_sacc[v + 1]):
            cnt = True
            curr_end = v + 1
        elif not pd.isnull(fast_sacc[v]) and pd.isnull(fast_sacc[v + 1]):
            # vels:
            total_vels.append(np.nansum(vels))
            total_v[starts[vels.index(np.nanmax(vels))]] = np.nansum(
                vels
            )  # rem
            # start
            start_idx.append(v + 1)
            start.append(time[v + 1])
            # end
            end_idx.append(curr_end)
            end.append(time[curr_end])
            interv[starts[vels.index(np.nanmax(vels))] + 1] = 600  # rem
            # interv[starts[vels.index(np.nanmax(vels))]] = 400 #rem
            # reset:
            cnt = False
            starts = []  # save the indicies in
            vels = []  # save the velocities in
        # if you are currently looking at intervals higher than the theshold: add them to the lists
        if (
            cnt == True
        ):  # and not pd.isnull(fast_sacc[v]) and pd.isnull(fast_sacc[v+1]):
            starts.append(v + 1)  # get the index
            vels.append(fast_sacc[v + 1])  # get the velocities

    end.append(time[-1])
    end_idx.append(len(fast_sacc))
    total_vels.append(np.nansum(vels))
    # interv[-1] = 400 #rem

    # order the data
    int_data = pd.DataFrame(
        list(zip(start, end, start_idx, end_idx, total_vels)),
        columns=["start", "end", "start_idx", "end_idx", "total_vels"],
    )

    # only get the big saccades
    max_initial_saccade_freq = 0.5
    # order the indicies according to the frequencies
    int_data = int_data.sort_values("total_vels", ascending=False)

    # if still smaller than max_initial_saccade_freq, keep adding nr to list, else, add 0
    new_start = [int_data["start"][0]]
    to_keep = [1]
    for st in int_data["start"][1:].tolist():
        new_start.append(st)
        new_start.sort()
        new_dist = [
            new_start[i + 1] - new_start[i] for i in range(len(new_start) - 1)
        ]
        if sum(new_dist) / len(new_dist) > max_initial_saccade_freq:
            to_keep = to_keep + [1]
        else:
            break

    to_keep = to_keep + [0] * (len(start) - len(to_keep))
    int_data["to_keep"] = to_keep
    int_data = int_data.sort_values("start_idx")

    # so we can make sure we will cover the entire duration
    to_keep = int_data["to_keep"].tolist()
    to_keep[0] = 1
    to_keep[-1] = 1
    int_data["to_keep"] = to_keep

    # now only save the list-elements we want to keep
    adjusted = True
    for v, val in enumerate(int_data["to_keep"].tolist()):
        if val == 0 and adjusted:
            curr_idx = v
            adjusted = False
        elif val == 1 and not adjusted:
            int_data.loc[v, "start"] = int_data["start"][curr_idx]
            int_data.loc[v, "start_idx"] = int_data["start_idx"][curr_idx]
            adjusted = True

    # delete all rows with 0
    int_data = int_data[int_data["to_keep"] == 1].reset_index()
    int_data = int_data.drop(columns=["to_keep"])

    int_data.to_csv(f"{PATH_FOREYE}/interval_mad_wobig_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

## 10 sec intervals (if you don't do the previous code)

In [50]:
# code comes from 5v_investigate_fixations
ids = recordings.index.tolist()
idd = ids[:]

int_len = 10  # number of seconds of the interval

for i, uid in enumerate(idd):
    for_eye = pd.read_csv(
        f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
    )
    time = for_eye["time"].tolist()

    start = []
    end = []
    start_idx = []
    end_idx = []
    # go through the dataframe and save the start and end each turn
    for t, ti in enumerate(time):
        if ti == time[0]:
            start.append(ti)
            start_idx.append(t)
        if ti - start[-1] > int_len:
            # if the current timepoint is more than int_len away from start, set it to new start
            start.append(ti)
            start_idx.append(t)
            # and set end to the timepoint before that
            end.append(time[t - 1])
            end_idx.append(t - 1)
    # add the last timepoint to end
    # (there is a very slim chance that the last start and end are the same timepoint --> might cause errors)
    end.append(time[-1])
    end_idx.append(len(time))

    # save it as new df
    int_data = list(zip(start, end, start_idx, end_idx))
    int_data = pd.DataFrame(
        int_data, columns=["start", "end", "start_idx", "end_idx"]
    )

    int_data.to_csv(f"{PATH_FOREYE}/interval_mad_10sec_{uid}.csv", index=True)

# Calculate MAD Saccade

## Get the thresholds for each interval

In [13]:
# go through the caclulated intervals, run at_mad on each of these intervals and save the thresholds
ids = recordings.index.tolist()
idd = ids[:]

for i, uid in enumerate(idd):
    display(uid)
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_10sec_{uid}.csv", index_col=0
        )

    combined_vel = for_eye["combined_vel"]
        
    # to shorten the slicing in the next for loop
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()

    # to add the final thresholds to for each segement
    scct = []
    for s, srt in enumerate(int_data["start"].tolist()):
        # get the slice of the combined velocity
        angular_vel = combined_vel[start_idx[s] : end_idx[s]]
        # use the at_mad function to caluclate the threshold
        saccade_th, thres = at_mad(angular_vel)
        if np.isnan(saccade_th):
            scct.append(thres[0])
        else:
            # add it to scct
            scct.append(saccade_th)

    # add it to int_data and save
    int_data["thresh"] = scct

    # display(len(scct))
    int_data = pd.DataFrame(int_data)
    # save
    # 1 == MAD_woBig
    if define_intervals == 1:
        int_data.to_csv(
            f"{PATH_FOREYE}/interval_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        int_data.to_csv(
            f"{PATH_FOREYE}/interval_mad_10sec_{uid}.csv", index=True
        )

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

C:\Users\schmi\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\schmi\anaconda3\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


## Add the threshold to for_eye df

In [14]:
# go through the caclulated intervals, run at_mad on each of these intervals and save the thresholds
ids = recordings.index.tolist()
idd = ids[:]

for i, uid in enumerate(idd):
    display(uid)
    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_10sec_{uid}.csv", index_col=0
        )

    time = for_eye.index.tolist()
    start_idx = int_data["start_idx"].tolist()
    end_idx = int_data["end_idx"].tolist()
    thr = int_data["thresh"].tolist()

    # go through all time intervals
    thresh = [0.0] * len(time)
    for s, srt in enumerate(int_data["start"].tolist()):
        # repeat the threshold as often as the time interval is long
        thresh = (
            thresh[: start_idx[s]]
            + [thr[s]] * len(time[start_idx[s] : end_idx[s]])
            + thresh[end_idx[s] :]
        )

    # add the two lists (ht & et) to for_eye df
    for_eye["thresh"] = thresh

    # save for_eye df
    for_eye = pd.DataFrame(for_eye)
    display(for_eye)
    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

,time,valid,eyeDirectionCombinedWorld.x,eyeDirectionCombinedWorld.y,eyeDirectionCombinedWorld.z,eyePositionCombinedWorld.x,eyePositionCombinedWorld.y,eyePositionCombinedWorld.z,eyeDirectionCombinedLocal.x,eyeDirectionCombinedLocal.y,...,HitPointOnObject.y,HitPointOnObject.z,hitObjectColliderName,hitObjectColliderName_all,ObjectColliderBoundsCenter.x,ObjectColliderBoundsCenter.y,ObjectColliderBoundsCenter.z,blinks,combined_vel,thresh
0,0.004,True,0.018,0.072,0.997,-59.101,2.418,34.760,-0.010,-0.003,...,6.931,97.609,Building_161,Building_161,-56.615,11.380,104.193,0.000,NaN,200.000
1,0.017,True,0.018,0.072,0.997,-59.101,2.418,34.760,-0.009,-0.004,...,6.941,97.611,NaN,Building_161,-56.615,11.380,104.193,0.000,NaN,200.000
2,0.023,True,0.018,0.071,0.997,-38.894,1.659,46.783,-0.009,-0.004,...,2.494,43.960,NaN,Building_161,-56.615,11.380,104.193,0.000,1000.000,0.000
3,0.032,True,0.018,0.072,0.997,-59.100,2.418,34.760,-0.008,-0.003,...,6.956,97.632,NaN,Building_161,-56.615,11.380,104.193,0.000,1000.000,0.000
4,0.043,True,0.019,0.072,0.997,-59.100,2.418,34.760,-0.007,-0.003,...,6.985,97.634,NaN,Building_161,-56.615,11.380,104.193,0.000,2.368,18.065
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159247,2019.309,False,-0.756,-0.126,-0.643,-157.930,0.125,-175.208,-0.226,-1.000,...,-0.681,-181.121,NaN,NaN,-163.089,-1.300,-184.651,0.000,0.000,0.000
159248,2019.320,False,-0.756,-0.126,-0.643,-157.930,0.125,-175.208,-0.226,-1.000,...,-0.681,-181.121,NaN,NaN,-163.089,-1.300,-184.651,0.000,0.000,0.000
159249,2019.331,False,-0.756,-0.126,-0.643,-157.930,0.125,-175.208,-0.226,-1.000,...,-0.681,-181.121,NaN,NaN,-163.089,-1.300,-184.651,0.000,0.000,0.000
159250,2019.342,False,-0.756,-0.126,-0.643,-157.930,0.125,-175.208,-0.226,-1.000,...,-0.681,-181.121,NaN,NaN,-163.089,-1.300,-184.651,0.000,0.000,0.000


## Define Fixations

In [15]:
ids = recordings.index.tolist()
idd = ids[:]

for i, uid in enumerate(idd):
    display(uid)
    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )
        int_data = pd.read_csv(
            f"{PATH_FOREYE}/interval_mad_10sec_{uid}.csv", index_col=0
        )

    start = int_data["start_idx"].tolist()
    end = int_data["end_idx"].tolist()
    thres = int_data["thresh"].tolist()
    combined_vel = for_eye["combined_vel"].tolist()

    # define list where the fixations will be added too
    is_fix = [np.nan] * len(combined_vel)

    for i in range(len(start)):
        av = combined_vel[start[i] : end[i]]
        # go through combined velocity and save all that are bigger than the threshold
        fix = [ti if ti < thres[i] else np.nan for ti in av]
        is_fix[start[i] : end[i]] = fix

    # save
    for_eye["isFix"] = is_fix

    # save data
    for_eye = pd.DataFrame(for_eye)

    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

## Correct small saccades and add gaze length + event onset/offset to list

In [16]:
##### CODE WORKING SO FAR
def correct_small_events(uid):
    """
    Correct is an event is too small.

    Parameters:
        uid (str): Participant identifier.
    """


    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    time = for_eye["time"].tolist()

    index = for_eye.index.tolist()  # index of df for easier use
    start_time = for_eye.iloc[0]["time"]  # update for each change
    start_idx = index[
        0
    ]  # will be updated each event and used to add to the lists

    # to save:
    isFix = []
    combined_vel = []

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_time = for_eye.iloc[1]["time"]  # update for each change
        start_idx = index[
            1
        ]  # will be updated each event and used to add to the lists
        isFix = [np.nan]
        combined_vel = [np.nan]

    # starting with a sacc
    if math.isnan(for_eye.loc[start_idx]["isFix"]):
        event = 0  # == sacc
    # starting with a gaze
    else:
        event = 1  # == gaze

    # hits progress bar
    index_pbar = tqdm(
        iterable=index[index.index(start_idx) : -1],
        desc=f"⌚ timestamps from participant {uid}",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )

    # display(for_eye[['time','isFix']])
    # go through the list:
    for idx in index_pbar:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx + 1]

        # gaze (--> sacc): now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
            if event == 1 and next_line.time - start_time < min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small and we are in big saccade, change isFix, change combined_vel
            elif event == 0 and next_line.time - start_time < min_gaze_dur:
                isFix = isFix + [np.nan] * (idx + 1 - start_idx)
                combined_vel = combined_vel + [np.nan] * (idx + 1 - start_idx)
            # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
            elif next_line.time - start_time >= min_gaze_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 1  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["time"]

        # sacc (--> gaze): now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
            if event == 0 and next_line.time - start_time < min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event to small but we are in big gaze, change isFix, change combined_vel
            elif event == 1 and next_line.time - start_time < min_sacc_dur:
                isFix = (
                    isFix
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # change isFix
                combined_vel = combined_vel + [np.nan] * (
                    idx + 1 - start_idx
                )  # change combined_vel
            # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
            elif next_line.time - start_time >= min_sacc_dur:
                isFix = (
                    isFix + for_eye.loc[start_idx:idx, "isFix"].values.tolist()
                )  # keep isFix
                combined_vel = (
                    combined_vel
                    + for_eye.loc[
                        start_idx:idx, "combined_vel"
                    ].values.tolist()
                )  # keep combined_vel
                event = 0  # change events
            # update start_time and start_idx
            start_idx = idx + 1
            start_time = for_eye.loc[idx + 1]["time"]

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                # if the event is too small but we are currently in a big gaze, keep isFix change combined_vel
                if (
                    event == 1
                    and next_line.time + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small and we are in big saccade, change isFix, change combined_vel
                elif (
                    event == 0
                    and next_line.time + 0.011 - start_time < min_gaze_dur
                ):
                    isFix = isFix + [np.nan] * (idx + 2 - start_idx)
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )
                # elif current event big enough, keep isFix and keep combined_vel and change event to 1,update length
                elif next_line.time + 0.011 - start_time >= min_gaze_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel
            # sacc:
            elif math.isnan(next_line.isFix):
                # if the event is too small and we are currently in a big sacc, keep isFix change combined_vel
                if (
                    event == 0
                    and next_line.time + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event to small but we are in big gaze, change isFix, change combined_vel
                elif (
                    event == 1
                    and next_line.time + 0.011 - start_time < min_sacc_dur
                ):
                    isFix = (
                        isFix
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # change isFix
                    combined_vel = combined_vel + [np.nan] * (
                        idx + 2 - start_idx
                    )  # change combined_vel
                # elif current event big enough, keep isFix and keep combined_vel and change event to 0,update length
                elif next_line.time + 0.011 - start_time >= min_sacc_dur:
                    isFix = (
                        isFix
                        + for_eye.loc[start_idx:, "isFix"].values.tolist()
                    )  # keep isFix
                    combined_vel = (
                        combined_vel
                        + for_eye.loc[
                            start_idx:, "combined_vel"
                        ].values.tolist()
                    )  # keep combined_vel

    # save everything:
    for_eye["isFix"] = isFix
    for_eye["corrected_vel"] = combined_vel
    # save data
    for_eye = pd.DataFrame(for_eye)

    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)



# participants ids
ids = recordings.index.tolist()
idd = ids[:]

# participants progress bar
parts_pbar = tqdm(
    iterable=idd,
    total=len(idd),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)


for i in parts_pbar:
    correct_small_events(i)


📄 0 of 1 📂 participants processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

📄 0 of 159251 ⌚ timestamps from participant 18d7a183-6a4a-4778-bd24-e74a388ffaaa processed:            
      …

## Define events, duration etc. 

In [17]:
def get_events_len_dist(uid):
    """
    Correct is an event is too small.

    Parameters:
        uid (str): Participant identifier.
    """


    min_sacc_dur = 0.02  # min sacc duration
    min_gaze_dur = 0.04  # min gaze duration (Ashima uses 0.05)

    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    time = for_eye["time"].tolist()


    ########## EVENTS, LENGTH, AVG DISTANCE, NAME OF OBJECT ##########
    index = for_eye.index.tolist()  # index of df for easier use

    events = [np.nan] * len(
        for_eye
    )  # sacc begin == 1, sacc end == -1; gaze begin == 2, gaze end == -2

    # if the first sample does not have any data
    if math.isnan(for_eye.iloc[0]["combined_vel"]) and not math.isnan(
        for_eye.iloc[1]["combined_vel"]
    ):
        start_idx = index[
            1
        ]  # will be updated each event and used to add to the lists
        events[1] = 2
        length = [np.nan]
        dist = [
            np.nan
        ]  # to save the distance to the hitpoint at each timestamps
        avg_dist = [
            np.nan
        ]  # to save the average distance of collider(s) during event
        names = [np.nan]  # to save the name of the current gaze
    else:
        start_idx = index[
            0
        ]  # will be updated each event and used to add to the lists
        length = []
        dist = []  # to save the distance to the hitpoint at each timestamps
        avg_dist = (
            []
        )  # to save the average distance of collider(s) during event
        names = []  # to save the name of the current gaze
        if math.isnan(for_eye.iloc[0]["combined_vel"]):
            events[0] = 1
        else:
            events[0] = 2
    start_time = for_eye.loc[start_idx]["time"].tolist()

    # hits progress bar
    index_pbar = tqdm(
        iterable=index[index.index(start_idx) : -1],
        desc=f"⌚ timestamps from participant {uid}",
        dynamic_ncols=True,
        bar_format=B_FORMAT,
    )
    # go through the list:
    for idx in index_pbar:
        curr_line = for_eye.loc[idx]
        next_line = for_eye.loc[idx + 1]

        # distance:
        hpoo = np.array(
            [curr_line["HitPointOnObject.x"], curr_line["HitPointOnObject.y"], curr_line["HitPointOnObject.z"]]
        )  # hitpoints on object
        coord_orig = np.array(
            [
                curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.x"],
                curr_line["eyePositionCombinedWorld.x"],
            ]
        )  # position of eyes
        dist = dist + [
            np.linalg.norm(hpoo - coord_orig)
        ]  # calculate to distance at this timpoint

        # gaze --> sacc: now gaze, next one is sacc
        if not math.isnan(curr_line.isFix) and math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "hitObjectColliderName_all"].values.tolist())
            )
            # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
            try:
                max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))
            except ValueError:
                max_keys = np.nan
            names = names + [max_keys] * (idx + 1 - start_idx)
            res = []
            
            # length, distance, events
            length = length + [curr_line.time - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx)] = -2  # end of gaze
            events[index.index(idx) + 1] = 1  # beginning of sacc
            # new idx
            start_time = curr_line.time
            start_idx = idx + 1

        # sacc --> gaze: now sacc, next one is gaze
        elif math.isnan(curr_line.isFix) and not math.isnan(next_line.isFix):
            # get name:
            res = dict(
                Counter(for_eye.loc[start_idx:idx, "hitObjectColliderName_all"].values.tolist())
            )
            # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
            try:
                max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))
                
            except ValueError:
                max_keys = np.nan
            names = names + [max_keys] * (idx + 1 - start_idx)
            res = []
            # length, distance, events
            length = length + [curr_line.time - start_time] * (
                idx + 1 - start_idx
            )  # length of event
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 1 - start_idx
            )  # average distance to collider(s) during event
            events[index.index(idx) + 1] = 2  # beginning of gaze
            ###### TO CHANGE POSSIBLY
            if events[index.index(idx)] != 1:
                events[index.index(idx)] = -1  # end of sacc
            # new idx
            start_time = curr_line.time
            start_idx = idx + 1

        # last index:
        if idx + 1 == index[-1]:
            # gaze:
            if not math.isnan(next_line.isFix):
                events[-1] = -2  # end of gaze
            # sacc:
            elif math.isnan(next_line.isFix):
                events[-1] = -1  # end of sacc
            length = length + [next_line.time - start_time] * (
                idx + 2 - start_idx
            )  # length of event
            # distance
            avg_dist = avg_dist + [
                np.nanmean(dist[index.index(start_idx) :])
            ] * (
                idx + 2 - start_idx
            )  # average distance to collider(s) during event
            hpoo = np.array(
                [next_line["HitPointOnObject.x"], next_line["HitPointOnObject.y"], next_line["HitPointOnObject.z"]]
            )  # hitpoints on object
            coord_orig = np.array(
                [
                    next_line["eyePositionCombinedWorld.x"],
                    next_line["eyePositionCombinedWorld.x"],
                    next_line["eyePositionCombinedWorld.x"],
                ]
            )  # position of eyes
            dist = dist + [
                np.linalg.norm(hpoo - coord_orig)
            ]  # calculate to distance at this timpoint
            # names
            res = dict(
                Counter(for_eye.loc[start_idx:, "hitObjectColliderName_all"].values.tolist())
            )
            # this case in case there is mostly nans but some names (so we don't get an nan in these cases)
            try:
                max_keys = max((key for key in res.keys() if isinstance(key, str)), key=(lambda new_k: res[new_k]))
            except ValueError:
                max_keys = np.nan
            names = names + [max_keys] * (idx + 2 - start_idx)
            res = []
            
        # display(len(names))
        # display(len(avg_dist))
    # save everything:
    for_eye["events"] = events
    for_eye["length"] = length
    for_eye["distance"] = dist
    for_eye["avg_dist"] = avg_dist
    for_eye["names"] = names
    # display(for_eye[['time','isFix','events','hon_all','names']])
    # save data
    for_eye = pd.DataFrame(for_eye)

    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)


# participants ids
ids = recordings.index.tolist()
idd = ids[:]

# participants progress bar
parts_pbar = tqdm(
    iterable=idd,
    total=len(idd),
    desc="📂 participants",
    dynamic_ncols=True,
    bar_format=B_FORMAT,
)

for i in parts_pbar:
    get_events_len_dist(i)


📄 0 of 1 📂 participants processed:            
              0% ⏱️00:00 ⏳? ⚙️?it/s

📄 0 of 159251 ⌚ timestamps from participant 18d7a183-6a4a-4778-bd24-e74a388ffaaa processed:            
      …

## Adjust the distance

In [18]:
# Change average distance to correct for the potential of other events
# so distance and avg_dist
# plot average duration of gazes and saccades
ids = recordings.index.tolist()
idd = ids[:]
for uid in idd:
    display(uid)
    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0)
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    # total lists:
    all_dist = []
    avg_dist = []
    # updated after each gaze
    dist = []
    hon_pos = []
    dur_gaze = False
    # during gaze:
    # go through the list:
    for g, gz in enumerate(for_eye["events"]):
        curr_line = for_eye.loc[g]
        if gz == 2.0 or gz == 1.0:
            dur_gaze = True
            # get the gazed at object
            curr_gaze = curr_line.names
        # if you are currently in a gaze:
        if dur_gaze:
            # if you are currently having the correct element, add the position
            if curr_line.hitObjectColliderName_all == curr_gaze:
                hon_pos = hon_pos + [
                    [
                        curr_line["HitPointOnObject.x"], curr_line["HitPointOnObject.y"], curr_line["HitPointOnObject.z"]
                    ]
                ]
            dist = dist + [
                np.array(
                    [
                        curr_line["eyePositionCombinedWorld.x"],
                        curr_line["eyePositionCombinedWorld.x"],
                        curr_line["eyePositionCombinedWorld.x"],
                    ]
                )
            ]

        # once the gaze is over, take the avg_dist
        if gz == -2.0 or gz == -1.0:
            hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
            # calculate to distance at this timpoint
            dist = [
                np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))
            ]
            all_dist = all_dist + dist
            # average distance during the gaze event
            avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

            # reset everything:
            dist = []
            hon_pos = []
            dur_gaze = False

        # if there are parts that are neither gaze nor saccade:
        if (
            (not dur_gaze)
            and (gz not in [2.0, 1.0])
            and (len(all_dist) + len(dist) != g + 1)
        ):
            all_dist = all_dist + [np.nan]
            avg_dist = avg_dist + [np.nan]

        if len(all_dist) + len(dist) != g + 1:
            display(g)

    if dur_gaze:
        hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
        # calculate to distance at this timpoint
        dist = [np.linalg.norm(hon_pos[c] - dist[c]) for c in range(len(dist))]
        all_dist = all_dist + dist
        # average distance during the gaze event
        avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)

    # add them to for_eye
    for_eye["distance"] = all_dist
    for_eye["avg_dist"] = avg_dist

    # save data
    for_eye = pd.DataFrame(for_eye)
    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True)
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1632520370.py:54: RuntimeWarning: Mean of empty slice
  hon_pos = [np.nanmean(hon_pos, axis=0)] * len(dist)
C:\Users\schmi\AppData\Local\Temp\ipykernel_18720\1632520370.py:61: RuntimeWarning: Mean of empty slice
  avg_dist = avg_dist + [np.nanmean(dist)] * len(dist)


# Length: Correct for long Events (outliers)

In [19]:
define_intervals

1

In [2]:
# perform median absolute deviation to find outliers in the event durations:
# https://hausetutorials.netlify.app/posts/2019-10-07-outlier-detection-with-median-absolute-deviation/

# Ashima:
# Finally, we rejected all fixations with a duration greater than 3.5 times the median absolute deviation of the
# population fixation duration...

ids = recordings.index.tolist()
idd = ids[:]

plotting = False
g_len = []
s_len = []

for uid in idd:
    display(uid)
    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )
    for_eye = for_eye[for_eye['valid'] == True]

    for_eye = for_eye[~for_eye["blinks"].isnull()]

    # separate between gaze and saccade
    gaze = for_eye[for_eye["events"] == 2.0]
    sacc = for_eye[
        for_eye["events"] == 1.0
    ]  # as we do not always have 1.0 in the data, we take the end

    g_len = g_len + gaze["length"].tolist()
    s_len = s_len + sacc["length"].tolist()

# median of the absolute deviation (* 1.4826 when assuming normal distribution):
gaze_mad = np.nanmedian(abs(g_len - np.nanmedian(g_len))) * 1.4826
sacc_mad = np.nanmedian(abs(s_len - np.nanmedian(s_len))) * 1.4826

# calculate the median to reject data
g_len_med = np.nanmedian(g_len)
s_len_med = np.nanmedian(s_len)

#
for uid in idd:
    # load data
    # 0 == MAD
    if define_intervals == 0:
        for_eye = pd.read_csv(f"{PATH_FOREYE}/correTS_{uid}.csv", index_col=0)
    # 1 == MAD_woBig
    elif define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    invalid = for_eye[for_eye['valid'] == False]
    for_eye = for_eye[for_eye['valid'] == True]

    # separate between gaze and saccade
    gaze = for_eye[~for_eye["isFix"].isnull()]
    sacc = for_eye[for_eye["isFix"].isnull()]

    # Deviations:
    # then to calculate the deviations using the entire df for easy lookup later
    gaze_mad_z = abs(gaze["length"].tolist() - g_len_med) / gaze_mad
    sacc_mad_z = abs(sacc["length"].tolist() - s_len_med) / sacc_mad

    # get the outliers:
    gaze_mad_z[gaze_mad_z > 3.5] = np.nan
    sacc_mad_z[sacc_mad_z > 3.5] = np.nan

    # save the data:
    for_eye = pd.concat([gaze, sacc, invalid])
    for_eye["long_events"] = gaze_mad_z.tolist() + sacc_mad_z.tolist() + [np.nan] * len(invalid)
    for_eye = for_eye.sort_index()

    # save df:
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye.to_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index=True
        )
    # 2 == 10_sec
    else:
        for_eye.to_csv(f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index=True)

NameError: name 'recordings' is not defined

In [10]:
ids = recordings.index.tolist()
idd = ids[:]
for uid in idd:
    display(uid)
    for_eye = pd.read_csv(f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0)
    gaze = for_eye[for_eye["events"] == 2.0]
    gaze = gaze[~gaze["long_events"].isnull()]

    gaze = gaze[~gaze["names"].isnull()]
    display(gaze)
    display(np.mean(gaze['length']))

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

,time,valid,eyeDirectionCombinedWorld.x,eyeDirectionCombinedWorld.y,eyeDirectionCombinedWorld.z,eyePositionCombinedWorld.x,eyePositionCombinedWorld.y,eyePositionCombinedWorld.z,eyeDirectionCombinedLocal.x,eyeDirectionCombinedLocal.y,...,combined_vel,thresh,isFix,corrected_vel,events,length,distance,avg_dist,names,long_events
4,0.043,True,0.019,0.072,0.997,-59.100,2.418,34.760,-0.007,-0.003,...,2.368,18.065,2.368,2.368,2.000,0.100,170.023,170.026,Building_161,0.768
17,0.187,True,0.043,0.043,0.998,-59.070,2.418,34.759,0.017,-0.029,...,12.917,18.065,12.917,12.917,2.000,0.188,170.187,170.203,Building_161,0.085
39,0.432,True,-0.043,0.018,0.999,-59.076,2.418,34.759,-0.073,-0.051,...,4.962,6.782,4.962,4.962,2.000,0.044,173.326,173.328,Building_161,1.202
135,1.489,True,-0.291,0.047,0.956,-59.094,2.418,34.759,-0.310,-0.014,...,2.972,7.057,2.972,2.972,2.000,0.310,187.506,187.512,CollisionObject1,0.861
173,1.911,True,-0.284,0.014,0.959,-59.101,2.418,34.759,-0.300,-0.047,...,2.174,7.057,2.174,2.174,2.000,0.288,189.241,189.240,CollisionObject0,0.690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159074,2017.395,True,-0.957,0.230,0.178,-158.796,0.129,-176.413,-0.070,0.251,...,2.303,20.658,2.303,2.303,2.000,0.312,169.987,170.006,TaskBuilding_7,0.876
159113,2017.829,True,-0.996,0.044,0.073,-159.602,0.142,-176.324,-0.017,0.116,...,4.382,9.506,4.382,4.382,2.000,0.066,162.421,162.543,Building_222,1.031
159137,2018.095,True,-0.982,-0.022,-0.189,-160.304,0.156,-176.306,-0.173,0.067,...,0.000,10.191,0.000,0.000,2.000,0.134,161.998,162.010,Wall_4,0.504
159159,2018.340,True,-0.779,-0.070,-0.622,-160.314,0.154,-176.308,-0.463,0.037,...,7.014,35.214,7.014,7.014,2.000,0.230,161.605,161.550,Wall_4,0.240


0.21397898592635484

# Generate TriggerFile + File with Additional Information for EEG Analysis

In [21]:
define_intervals

1

In [22]:
# streams to remove
REMOVE = [
    "openvibeSignal",
    "openvibeMarkers",
    "Diode",
    "TimeStampBeginlsl",
    "TimeStampEndlsl",
    "TrialNumber",
    "CancelPressed",
    "handPosDirRot", # Can be excluded for explorations but must be included for task sessions
    "bodyTrackerPosRot", # Same here. Include for task sessions, exclude for explorations (use HMD positions instead)
    "TriggerPressed",
]

In [8]:
Global_Landmark = ['crane_1', 'crane_2', 'Castle-TaskBuilding_56','HighSilo-TaskBuilding_49', 'House_49', 
                   'Windmill-TaskBuilding_10_1', 'House_10', 'Church-TaskBuilding_16', 'church'] 

TaskBuilding_Public = ['TaskBuilding_2','TaskBuilding_3', 'TaskBuilding_5', 'TaskBuilding_8', 'Graffity_2', 'Graffity_3', 
                       'Graffity_5', 'Graffity_8', 'TaskBuilding_46', 'Building_46', 'Garage_46', 'TaskBuilding_9',
                       'TaskBuilding_11',
                       'TaskBuilding_13', 'TaskBuilding_14', 'TaskBuilding_20', 'TaskBuilding_21', 'TaskBuilding_23', 
                       'TaskBuilding_27', 'TaskBuilding_29', 'TaskBuilding_34', 'TaskBuilding_38', 'TaskBuilding_41', 
                       'TaskBuilding_42', 'TaskBuilding_44', 'TaskBuilding_47', 'TaskBuilding_50', 'TaskBuilding_52', 
                       'TaskBuilding_45', 'TaskBuilding_53', 'Task_Building_32', 'Graffity_9', 'Graffity_11', 
                       'Graffity_13', 'Graffity_14', 'Graffity_20', 'Graffity_21', 'Graffity_23', 'Graffity_27', 
                       'Graffity_29', 'Graffity_34', 'Graffity_38', 'Graffity_41', 'Graffity_42', 'Graffity_44', 
                       'Graffity_47', 'Graffity_50', 'Graffity_52', 'Graffity_45', 'Graffity_53', 'Graffity_32']

TaskBuilding_Residential = ['TaskBuilding_1','TaskBuilding_4', 'TaskBuilding_6', 'TaskBuilding_7',  'Graffity_1', 
                            'Graffity_4', 'Graffity_6', 'Graffity_7', 'Garage_26', 'TaskBuilding_12', 'TaskBuilding_15',
                            'TaskBuilding_18','TaskBuilding_17','TaskBuilding_19','TaskBuilding_22','TaskBuilding_24',
                            'TaskBuilding_26', 'TaskBuilding_28','TaskBuilding_31','TaskBuilding_33','TaskBuilding_35',
                            'TaskBuilding_36','TaskBuilding_37','TaskBuilding_39','TaskBuilding_48','TaskBuilding_25',
                            'TaskBuilding_54','TaskBuilding_55','Graffity_12', 'Graffity_15', 'Graffity_18', 
                            'Graffity_17', 'Graffity_19', 'Graffity_22', 'Graffity_24', 'Graffity_26', 'Graffity_28', 
                            'Graffity_31', 'Graffity_33', 'Graffity_35', 'Graffity_36', 'Graffity_37', 'Graffity_39', 
                            'Graffity_48', 'Graffity_25', 'Graffity_54', 'Graffity_55']

Other_Buildings = ['Building_57', 'Building_58', 'Building_59', 'Building_60', 'Building_61', 'Building_62', 'Building_63',
                   'Building_64', 'Building_65', 'Building_66', 'Building_67', 'Building_68', 'Building_69', 'Building_70',
                   'Building_71', 'Building_72', 'Building_73', 'Building_74', 'Building_75', 'Building_76', 'Building_77',
                   'Building_78', 'Building_79', 'Building_80', 'Building_81', 'Building_82', 'Building_83', 'Building_84',
                   'Building_85', 'Building_86', 'Building_87', 'Building_88', 'Building_89', 'Building_90', 'Building_91',
                   'Building_92', 'Building_93', 'Building_94', 'Building_95', 'Building_96', 'Building_97', 'Building_98',
                   'Building_99', 'Building_100', 'Building_101', 'Building_102', 'Building_103', 'Building_104', 
                   'Building_105', 'Building_106', 'Building_107', 'Building_108', 'Building_109', 'Building_110', 
                   'Building_111', 'Building_112', 'Building_113', 'Building_114', 'Building_115', 'Building_116', 
                   'Building_117', 'Building_118', 'Building_119', 'Building_120', 'Building_121', 'Building_122', 
                   'Building_123', 'Building_124', 'Building_125', 'Building_126', 'Building_127', 'Building_128', 
                   'Building_129', 'Building_130', 'Building_131', 'Building_132', 'Building_133', 'Building_134', 
                   'Building_135', 'Building_136', 'Building_137', 'Building_138', 'Building_139', 'Building_140', 
                   'Building_141', 'Building_142', 'Building_143', 'Building_144', 'Building_145', 'Building_146', 
                   'Building_147', 'Building_148', 'Building_149', 'Building_150', 'Building_151', 'Building_152', 
                   'Building_153', 'Building_154', 'Building_155', 'Building_156', 'Building_157', 'building01_LOD0', 
                   'building01_LOD1', 'Building_158', 'Building_159', 'Building_160', 'Building_161', 'Building_162', 
                   'Building_163', 'Building_164', 'Building_165', 'Building_166', 'Building_167', 'Building_168', 
                   'Building_169', 'Building_170', 'Building_171', 'building02_LOD0', 'Building_172', 'Building_173', 
                   'Building_174', 'Building_175', 'Building_176', 'Building_177', 'Building_178', 'Building_179', 
                   'Building_180', 'Building_181', 'Building_182', 'Building_183', 'Building_184', 'Building_185', 
                   'Building_186', 'Building_187', 'Building_188', 'Building_189', 'Building_190', 'Building_191', 
                   'Building_192', 'Building_193', 'Building_194', 'Building_195', 'Building_196', 'Building_197', 
                   'Building_198', 'Building_199', 'Building_200', 'Building_201', 'Building_202', 'Building_203', 
                   'Building_204', 'Building_205', 'Building_206', 'Building_207', 'Building_208', 'Building_209', 
                   'Building_210', 'Building_211', 'Building_212', 'Building_213', 'Building_214', 'Building_215', 
                   'Building_216', 'Building_217', 'Building_218', 'Building_219', 'Building_220', 'Building_221', 
                   'Building_222', 'Building_223', 'Building_224', 'Building_225', 'Building_226', 'Building_227', 
                   'Building_228', 'Building_229', 'Building_230', 'Building_231', 'Building_232', 'Building_233', 
                   'Building_234', 'Building_235', 'Building_236', 'Garage_71', 'Garage_72', 'Garage_86', 'Garage_98', 
                   'Garage_185', 'Garage_224', 'Garage_235']

In [24]:
def compute_centroid(df, columns):
    centroid = df[columns].mean().values
    return centroid

def compute_saccade_amplitude(prev_gaze_centroid, prev_subject_centroid, gaze_centroid, subject_centroid):
    v_eye_vec = gaze_centroid - prev_gaze_centroid
    eye_vec = prev_gaze_centroid - prev_subject_centroid
    eye_vec = eye_vec/np.linalg.norm(eye_vec)
    projection = np.dot(v_eye_vec, eye_vec) * eye_vec
    v_eye_inplane = np.linalg.norm(v_eye_vec - projection)
    sacc_amplitude = np.arctan2(v_eye_inplane, np.linalg.norm(prev_subject_centroid - prev_gaze_centroid))
    return np.degrees(sacc_amplitude)

In [25]:
# participants ids
ids = recordings.index.tolist()
idd = ids[:]


for uid in idd:
    display(uid)
    # load raw data
    part = recordings.loc[uid].file
    data, _ = pyxdf.load_xdf(f"{PATH_RAW}/{part}")

    starts = []
    names = []
    ends = []
    # get the new time:
    for s in data:
        # stream name
        s_name = s["info"]["name"][0]
        # check SR of the EEG data
        if "openvibeSignal" in s_name:
            time_eeg = s["time_stamps"][0]
            time_eeg_end = s["time_stamps"][-1]
        elif s_name not in REMOVE:
            starts.append(s["time_stamps"][0])
            names.append(s_name)
            ends.append(s["time_stamps"][-1])
        if "EyeTrackingPosDir" in s_name:
            et_start = s["time_stamps"][0]

    # check if P100 will be delayed, then double check if we have additional delays/shifts to take care off
    to_sub = et_start - min(starts)  # this is done to correct for the onset
    start = min(starts) - time_eeg
    # TBD: decide which end to use!!!
    # end = ends[starts.index(min(starts))] - time_eeg_end
    end = max(ends) - time_eeg_end

    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )

    # correct unity with the different timestamps
    time = for_eye["time"].tolist() + to_sub
    time = [round(t, 3) for t in time]

    # interpolate the timestamps but save previous version
    ts = np.linspace(time[0], time[-1], num=int(time[-1] * 1000))
    ts = [round(t, 3) for t in ts]
    ts = list(dict.fromkeys(ts))  # remove dublicates

    # add shift
    dynamic_shift = end - start
    drift_correction_np = np.linspace(
        0, dynamic_shift, len(ts)
    )  # np.linspace(start,end,num)
    # linearly correct the timestamps from start to end given the drift increase over time 'dynamic_shift'
    ts_shift = (ts - drift_correction_np).tolist()

    # take interpolated ts out
    time_df = pd.DataFrame(
        list(zip(ts, ts_shift)), columns=["time", "time_shift"]
    )
    time_df = time_df[time_df["time"].isin(time)]

    # exchange the times in for_eye
    for_eye["time"] = time_df["time_shift"].tolist()
    
    
     # --- SACCADE AMPLITUDE ---
    prev_gaze_centroid = None
    prev_subject_centroid = None
    for_eye['saccade_amplitude'] = np.nan
    start_indices = for_eye[for_eye['events'] == 2.0].index
    end_indices = for_eye[for_eye['events'] == -2.0].index
    if len(end_indices) > len(start_indices):
        if end_indices[0] < start_indices[0]:
            end_indices = end_indices[1:]
        else: 
            print(uid)
    for start, end in zip(start_indices, end_indices):
        group_df = for_eye.loc[start:end]
        # Compute the centroid of the gaze positions and the mean timestamp
        gaze_centroid = compute_centroid(group_df, ['HitPointOnObject.x', 'HitPointOnObject.y', 'HitPointOnObject.z'])
        subject_centroid = compute_centroid(group_df, ['eyePositionCombinedWorld.x', 'eyePositionCombinedWorld.y', 
                                                       'eyePositionCombinedWorld.z'])
        # Compute the saccade amplitude and store it in the dataframe
        if prev_gaze_centroid is not None and prev_subject_centroid is not None:
            for_eye.loc[start:end, 'saccade_amplitude'] = compute_saccade_amplitude(prev_gaze_centroid, 
                                                                                    prev_subject_centroid, 
                                                                                    gaze_centroid, subject_centroid)
        # Update the previous centroids and timestamp
        prev_gaze_centroid = gaze_centroid
        prev_subject_centroid = subject_centroid
    # --- SACCADE AMPLITUDE ---
    
    
    
    # create triggers as done in the code before
    # variables
    gaze = for_eye[for_eye["events"] == 2.0]
    gaze = gaze[~gaze["long_events"].isnull()]

    no_name = gaze[gaze["names"].isnull()]
    gaze = gaze[~gaze["names"].isnull()]
    ### VS: It could be that a gaze contains a large number of NaNs and one hit point, based on the code
    ### above, the gaze could be set to the one hit point --> check?!
    
   # now we define the different categories for the different triggers
    Global_Landmark_df = gaze[gaze["names"].isin(Global_Landmark)]
    TaskBuilding_Public_df = gaze[gaze["names"].isin(TaskBuilding_Public)]
    TaskBuilding_Residential_df = gaze[gaze["names"].isin(TaskBuilding_Residential)]
    Other_Buildings_df = gaze[gaze["names"].isin(Other_Buildings)]
    Background_df = gaze[~gaze["names"].isin(Global_Landmark+TaskBuilding_Public+TaskBuilding_Residential+
                                             Other_Buildings)] # '~' inverts the statement
    
    # create the trigger df:
    # first get all the starts in one column
    lat = pd.concat([Global_Landmark_df["time"], TaskBuilding_Public_df["time"], TaskBuilding_Residential_df["time"], 
                     Other_Buildings_df["time"], Background_df["time"], no_name["time"]])
    lat = pd.DataFrame(lat)

    # then add the type: 2 for background, 1 for NPCs and 0 for faces, 3 for no name
    lat["type"] = (
        ["Global_Landmark"] * len(Global_Landmark_df["time"]) +
        ["TaskBuilding_Public"] * len(TaskBuilding_Public_df["time"]) + 
        ["TaskBuilding_Residential"] * len(TaskBuilding_Residential_df["time"]) + 
        ["Other_Buildings"] * len(Other_Buildings_df["time"]) + 
        ["Background"] * len(Background_df["time"]) + 
        ["no_name"] * len(no_name["time"])
    )
    
    # sort it by start so that all the events are in correct order
    lat = lat.sort_values(by=["time"])

    # rename start to latency
    lat = lat.rename(columns={"time": "latency"})
    lat["saccade_amp"] = gaze["saccade_amplitude"]
    
    # 1 == MAD_woBig
    if define_intervals == 1:
        lat.to_csv(f"{PATH_TRG}/TriggerFile_dd_{uid}.csv", index=False)
    # 2 == 10_sec
    else:
        lat.to_csv(f"{PATH_TRG}/TriggerFile_10_{uid}.csv", index=False)

'18d7a183-6a4a-4778-bd24-e74a388ffaaa'

## New Trigger Files without Latency Corrections

In [10]:
# participants ids
ids = recordings.index.tolist()
idd = ids[:]


for uid in idd:
    # load data
    # 1 == MAD_woBig
    if define_intervals == 1:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS_mad_wobig_{uid}.csv", index_col=0
        )
    # 2 == 10_sec
    else:
        for_eye = pd.read_csv(
            f"{PATH_FOREYE}/correTS__10sec_{uid}.csv", index_col=0
        )
    
    
    
    # create triggers as done in the code before
    # variables
    gaze = for_eye[for_eye["events"] == 2.0]
    gaze = gaze[~gaze["long_events"].isnull()]

    no_name = gaze[gaze["names"].isnull()]
    gaze = gaze[~gaze["names"].isnull()]
    ### VS: It could be that a gaze contains a large number of NaNs and one hit point, based on the code
    ### above, the gaze could be set to the one hit point --> check?!
    
   # now we define the different categories for the different triggers
    Global_Landmark_df = gaze[gaze["names"].isin(Global_Landmark)]
    TaskBuilding_Public_df = gaze[gaze["names"].isin(TaskBuilding_Public)]
    TaskBuilding_Residential_df = gaze[gaze["names"].isin(TaskBuilding_Residential)]
    Other_Buildings_df = gaze[gaze["names"].isin(Other_Buildings)]
    Background_df = gaze[~gaze["names"].isin(Global_Landmark+TaskBuilding_Public+TaskBuilding_Residential+
                                             Other_Buildings)] # '~' inverts the statement
    
    # create the trigger df:
    # first get all the starts in one column
    lat = pd.concat([Global_Landmark_df["time"], TaskBuilding_Public_df["time"], TaskBuilding_Residential_df["time"], 
                     Other_Buildings_df["time"], Background_df["time"], no_name["time"]])
    lat = pd.DataFrame(lat)

    # then add the type: 2 for background, 1 for NPCs and 0 for faces, 3 for no name
    lat["type"] = (
        ["Global_Landmark"] * len(Global_Landmark_df["time"]) +
        ["TaskBuilding_Public"] * len(TaskBuilding_Public_df["time"]) + 
        ["TaskBuilding_Residential"] * len(TaskBuilding_Residential_df["time"]) + 
        ["Other_Buildings"] * len(Other_Buildings_df["time"]) + 
        ["Background"] * len(Background_df["time"]) + 
        ["no_name"] * len(no_name["time"])
    )
    
    # sort it by start so that all the events are in correct order
    lat = lat.sort_values(by=["time"])

    # rename start to latency
    lat = lat.rename(columns={"time": "latency"})
    
    # 1 == MAD_woBig
    if define_intervals == 1:
        #lat.to_csv(f"{PATH_TRG}/TriggerFile_dd_{uid}.csv", index=False)
        lat.to_csv(f"{PATH_TRG}/TriggerFile_wo_latency_correction_{uid}.csv", index=False)
    # 2 == 10_sec
    else:
        lat.to_csv(f"{PATH_TRG}/TriggerFile_10_{uid}.csv", index=False)

,latency,type
4,0.043,Other_Buildings
17,0.187,Other_Buildings
39,0.432,Other_Buildings
135,1.489,Background
173,1.911,Background
...,...,...
159074,2017.395,TaskBuilding_Residential
159113,2017.829,Other_Buildings
159137,2018.095,Background
159159,2018.340,Background
